## Animating tesseract rotations via 3D Mobius transformations

$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\th}{\theta}$
$\newcommand{\lam}{\lambda}$
$\newcommand{\ovl}{\overline}$
$\newcommand{\ds}{\displaystyle}$
$\newcommand{\barr}{\begin{array}}$
$\newcommand{\earr}{\end{array}}$
$\newcommand{\bea}{\begin{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$


Tesseract [https://en.wikipedia.org/wiki/Tesseract](https://en.wikipedia.org/wiki/Tesseract) is a hypercube in $\R^4$. It's 3D version is represented  by a cube and a translated scaled version of it.
Let us animate the rotation of tesseract about each coordinate plane in $\R^4$:  'xy`, 'xz', 'yz', 'xu' 'yu', 'zu', as well as about an arbitrary plane containing the origin, O, of the system of coordinates $O; x, y, z, u$:

In [1]:
import numpy as np
from numpy import pi, sin, cos
import json
import plotly.graph_objs as go
from mobius3d import *

from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

### Tesseract rotation about a plane of coordinates in $\R^4$

Read the json file containing the 3D tesseract vertices and edges:

In [2]:
with open('Data/tesseract.json', 'r') as fp:
    jdata = json.load( fp)

vertices = np.array([node['pos'] for node in jdata['nodes']])
edges = jdata['edges']

Define two Plotly traces that represent the internal, respectively the external tesseract cube:

In [3]:
Xe = []
Ye = []
Ze = []

for e in edges[:12]:
    Xe.extend([vertices[e[0], 0], vertices[e[1], 0], None])
    Ye.extend([vertices[e[0], 1], vertices[e[1], 1], None])
    Ze.extend([vertices[e[0], 2], vertices[e[1], 2], None])
fig = go.Figure(go.Scatter3d(x = Xe, y=Ye, z=Ze, 
                              mode='lines', name= 'initially-inside',
                              line_width=10, line_color='red'))  

Xe = []
Ye = []
Ze = []
for e in edges[12:]:
    Xe.extend([vertices[e[0], 0], vertices[e[1], 0], None])
    Ye.extend([vertices[e[0], 1], vertices[e[1], 1], None])
    Ze.extend([vertices[e[0], 2], vertices[e[1], 2], None])
fig.add_scatter3d(x = Xe, y=Ye, z=Ze, 
                   mode='lines', name='initialy-outside', 
                   line_width=10, line_color='RoyalBlue'); 

Functions that defines the frames for animating  a particular rotation:

In [4]:
def get_frame_trace(rvertices, edges):
    #rvertices - rotated tesseract vertices
    Xe = []
    Ye = []
    Ze = []

    for e in edges: 
        Xe.extend([rvertices[e[0], 0], rvertices[e[1], 0], None])
        Ye.extend([rvertices[e[0], 1], rvertices[e[1], 1], None])
        Ze.extend([rvertices[e[0], 2], rvertices[e[1], 2], None])
    
    return go.Scatter3d(x=Xe, y=Ye, z=Ze)    

Define below the animation frames for rotation about `'xy'`, `'xz'`, `'yz'`, `'xu'`, `'yu'`, `'zu'`. The default rotation is about `'xy'`, represented by the matrix $R_1$,
defined in [](). For any other rotation,  one identifies the orthogonal matrix $U$, in its orthogonal decomposition. In each case the matrix $U$ is a permutation matrix [https://en.wikipedia.org/wiki/Permutation_matrix](https://en.wikipedia.org/wiki/Permutation_matrix).

For example the rotation about the plane, `'xz'`, has the representative matrix:

$$R_{xz} =\left(\barr{cccc} 1&0&0&0\\0&\cos(t)&0&\sin(t)\\ 0&0&1&0\\0&-\sin(t)&0&\cos(t)\earr\right)$$
which is related to the matrix, $R_1$, of rotation about $`'xy'`, by the permutation matrix associated to the permutation
$$\sigma=\left(\barr{cccc} 0&1&2&3\\
0&2&1&3\earr\right)$$
i.e. the matrix $P_\sigma$ obtained from the identity matrix by permuting its rows according to $\sigma$:

$R_{xz} = P_\sigma R_{xy}P_\sigma^T$. Hence we get the  rotation $R_{xz}$ from $R_{xy}$, by permuting both rows and columns in $R_{xy}$ according to the permutation $\sigma$. But a permutation matrix is an orthogonal matrix, and as a consequence this last relation
is the orthogonal decomposition of the rotation $R_{xz}$. Below we are giving the corresponding permutation matrix for each particular rotation about a coordinate plane in $\R^4$:

In [5]:
def get_frames(vertices,  theta, s=0,  about='xy'):
    # vertices -  array of tesseract vertex coordinates
    #theta - array of shape (n, ) of consecutive angles of rotation, during the animation
    # returns  frames for tesseract animation
    # NOTE that the angle of rotation s, in R_2, is fixed on 0, to get R_1
    frames = []
    Id = np.eye(4)
    if about == 'xy':
        perm = [0, 1, 2, 3]
    elif about == 'xz':
        perm = [0, 2, 1, 3]   
    elif about == 'yz':
        perm = [2, 0, 1, 3]  
    elif about ==  'xu':
        perm = [0, 2, 3, 1]
    elif about == 'yu':
        perm = [2, 0, 3, 1]
    elif about == 'zu':
        perm = [2, 3, 0, 1]
    else:
        raise ValueError('Bad name for the plane the rotation is about')
    U = Id[perm, :]    
    for t in theta:
        Rot = setup_rotation(s=s, t=t, U=U)
        #get tesseract vertices after Mobius transf corresponding to rotation Rot
        rvertices = (Mobius3d_trans(vertices.T,  Rot=Rot)).T 
        tr1 = get_frame_trace(rvertices, edges[:12])
        tr2 = get_frame_trace(rvertices, edges[12:])
    
        frames.append(go.Frame(data=[tr1, tr2],
                               traces=[0,1],
                               group = about))
    return frames  

In [6]:
#Fix the camera eye position:
x = 2
y = 2
z = x+1j*y
w = np.exp(1j*pi/12)*z #camera eye position

fig.update_scenes(xaxis_visible=False,
                   yaxis_visible=False,
                   zaxis_visible=False,
                   aspectmode='data',
                   camera_eye=dict(x=w.real, y=w.imag, z=0.95));

In [7]:
fig.update_layout(width=600, height=600,
                   updatemenus=[dict(type='buttons', 
                                y=0.8,
                                x=1.2,
                                showactive = False,    
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=30, 
                                                                    redraw=True),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')]),
                                         dict(label='Pause',
                                              method='animate',
                                              args=[None, 
                                                    dict(frame=dict(duration=0, 
                                                                    redraw=False),
                                                         transition=dict(duration=0),
                                                         fromcurrent=True,
                                                         mode='immediate')])
                                        ])]);


To animate a 360-degree rotation just run successively the following cell, changing `about` string with one from the list:

`['xy', 'xz', 'yz', 'xu', 'yu', 'zu']`:

In [8]:
theta = np.linspace(0, 2*pi, 72) # the default direction of rotation resulted from matrix C_1 

#OR

theta = np.linspace(2*pi, 0, 72) #opposite direction of rotation

rot_about = 'xy'
frames = get_frames(vertices, theta, about=rot_about)

fig.update(frames=frames); 
fig.update_layout(title_text=f'Tesseract rotation about the plane {rot_about}', title_x=0.5)
iplot(fig)

### Tesseract double rotation animation

In [9]:
n_rotations = 72
s_values = np.linspace(0, 2*pi, n_rotations)
t_values =  - np.linspace(0, 2*pi, n_rotations)
frames = []
for k in range(n_rotations):
    Rot = setup_rotation(s=s_values[k], t= t_values[k], U=np.eye(4)) #double rotation of angle s[k]  and t[k]=-s[k]
    rvertices = (Mobius3d_trans(vertices.T,  Rot=Rot)).T 
    tr1 = get_frame_trace(rvertices, edges[:12])
    tr2 = get_frame_trace(rvertices, edges[12:])
    
    frames.append(go.Frame(data=[tr1, tr2],
                           traces=[0,1]))
        
fig.update(frames=frames); 
fig.update_layout(title_text=f'Tesseract double rotation animation')

iplot(fig)            

### Tesseract rotation about an arbitrary plane through the origin, `O`

Let us deduce the rotation  about an arbitrary  4D subspace spanned by two vectors $w1, w2$:

Consider three  vertices, whose plane in $\R^3$  is a diagonal plane of the tesseract, and map them onto the 3d sphere, via the inverse stereographic projection:

In [10]:
A = (0.75, -0.75,-0.75)
B = (0.75, -0.75, 0.75)
C = (-0.75, 0.75, -0.75)
point3d = np.stack((A, B, C), axis=-1)

In [11]:
sph_point = inv_stereo_aS3(point3d).T

Define the rotation matrix about the subspace in $\R^4$ spanned by two independent vectors, w1, w2

In [12]:
w1 = sph_point[1]-sph_point[0]
w2 = sph_point[2]-sph_point[0]

The function `ort_gen_rotation`, from `moubius3d`, defines an orthonormal basis, `(u1, u2)`,  in the subspace `span(w1, w2)`. These two vectors are columns in the Q component of the QR-decomposition of the 
matrix, $A$,  having w1, w2 as rows.  Then,   `scipy.linalg.null_space(A)` returns  an orthonormal basis, `(u3, u4)`
in the orthogonal complement of the subspace spanned by `(u1, u2)`. The matrix $U$` having as columns the vectors u1, u2, u3, u4` is the orthogonal matrix in the orthogonal decomposition of the rotation $Q$ about the subspace `span(u1, u2)` = `span(w1, w2)`:


In [13]:
U = ort_gen_rotation(w1, w2)

In [14]:
np.linalg.norm(U @ U.T -np.eye(4)) #check how close is U^TU to Id_4

3.998090802177286e-16

In [15]:
theta = np.linspace(2*pi, 0, 72)
frames = []
for angle in theta:
    s=0
    Rot = setup_rotation(s=0, t=angle, U=U)
    #get tesseract vertices after Mobius transf corresponding to rotation Rot
    rvertices = (Mobius3d_trans(vertices.T,  Rot=Rot)).T 
    tr1 = get_frame_trace(rvertices, edges[:12])
    tr2 = get_frame_trace(rvertices, edges[12:])
    
    frames.append(go.Frame(data=[tr1, tr2],
                           traces=[0,1],
                               ))   

In [16]:
fig.update(frames=frames); 
fig.update_layout(title_text=f'Tesseract rotation about an arbitrary subspace')
iplot(fig)